In [1]:
!pip install torch torchvision scikit-learn pandas matplotlib tqdm gradio
!pip install kagglehub

In [2]:
# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
from torchvision.datasets import DatasetFolder, ImageFolder

# Scikit-learn
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

# Misc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import kagglehub

# Subject to change
import gradio as gr

In [3]:
data_dir = kagglehub.dataset_download("bharathry/glaucoma-dataset-for-ai-diagnosis")
print("Downloaded at:", data_dir)

data_dir = os.path.join(data_dir, "Glaucoma Dataset for AI Diagnosis")
# Checking file structure as I made the dataset with more nested folders than I needed to
print("Current Data Dir:", data_dir)
print("Top-Level Folders:", os.listdir(data_dir))

100%|██████████| 2.67G/2.67G [02:49<00:00, 16.9MB/s]

Extracting files...


Downloaded at: /Users/bharathreddy/.cache/kagglehub/datasets/bharathry/glaucoma-dataset-for-ai-diagnosis/versions/1
Current Data Dir: /Users/bharathreddy/.cache/kagglehub/datasets/bharathry/glaucoma-dataset-for-ai-diagnosis/versions/1/Glaucoma Dataset for AI Diagnosis
Top-Level Folders: ['test', 'train', 'val']


In [4]:
IMG_SIZE = 224
BATCH_SIZE = 32

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [5]:
from PIL import Image

# Function to find images in a folder and label them because the dataset is structured with 
# subfolders for each class making it harder to use the Images normally
def find_images(folder_path, label):
    images = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith((".jpg", ".jpeg", ".png")):
                images.append((os.path.join(root, file), label))
    return images

train_images = (
    find_images(os.path.join(data_dir, "train", "glaucoma"), 1) +
    find_images(os.path.join(data_dir, "train", "normal"), 0)
)

val_images = (
    find_images(os.path.join(data_dir, "val", "glaucoma"), 1) +
    find_images(os.path.join(data_dir, "val", "normal"), 0)
)

# Print the number of images found in each set
print(f"Found {len(train_images)} train images, {len(val_images)} val images")


Found 2590 train images, 170 val images


In [6]:
# Class for custom dataset

class CustomDataset(Dataset):
    def __init__(self, image_list, transform=None):
        self.image_list = image_list
        self.transform = transform # Transformations to apply to the images

    def __len__(self):
        return len(self.image_list) 

    def __getitem__(self, index):
        img_path, label = self.image_list[index]
        # image = Image.open(img_path).convert("RGB") # (Optional) Convert to RGB in case of grayscale images, I am not using this because there aren't grayscale images in the dataset
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
from sklearn.model_selection import train_test_split

train_idx, test_idx = train_test_split(
    range(len(train_images)),
    test_size=0.2,
    stratify=[label for x, label in train_images],
    random_state=42
)

train_split = [train_images[i] for i in train_idx]
test_split = [train_images[i] for i in test_idx]

print(f"Final counts — Train: {len(train_split)}, Test: {len(test_split)}, Val: {len(val_images)}")

Final counts — Train: 2072, Test: 518, Val: 170


In [8]:
train_dataset = CustomDataset(train_split, transform=transform)
test_dataset = CustomDataset(test_split, transform=transform)
val_dataset = CustomDataset(val_images, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)